In [2]:
#Here's another example of potential GpyTorch implementation of Cocabo

import torch
import gpytorch
import botorch
import numpy as np
from gpytorch.kernels import MaternKernel, Kernel
from botorch.models import SingleTaskGP
#from botorch.fit import fit_gpytorch_model
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf

# Custom Kernel: Matern for continuous + Learned Similarity for categorical
class CategoricalMaternKernel(Kernel):
    def __init__(self, num_categories, ard_num_dims=None, **kwargs):
        super().__init__(**kwargs)
        self.matern = MaternKernel(nu=2.5, ard_num_dims=3)  # 3 continuous vars
        self.similarity_matrix = torch.nn.Parameter(
            torch.rand(num_categories, num_categories)
        )  # Trainable similarity matrix
        self.lambda_param = torch.nn.Parameter(torch.tensor(0.5))  # Weighting param

    def forward(self, x1, x2, diag=False, **params):
        x1_cont, x1_cat = x1[..., :3], x1[..., 3].long()
        x2_cont, x2_cat = x2[..., :3], x2[..., 3].long()
        
        k_x = self.matern(x1_cont, x2_cont, diag=diag, **params)
        k_h = self.similarity_matrix[x1_cat, x2_cat]
        
        return self.lambda_param * k_x + (1 - self.lambda_param) * k_h

# GP Model
class CustomGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_categories):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = CategoricalMaternKernel(num_categories)
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Multi-Armed Bandit (UCB for categorical selection)
def select_categorical_option(similarity_matrix, categorical_values, beta=2.0):
    scores = similarity_matrix.mean(dim=1) + beta * similarity_matrix.std(dim=1)
    return categorical_values[torch.argmax(scores)]

# Bayesian Optimization Loop
def bayesian_optimization_loop(train_x, train_y, num_iters=10, beta=2.0):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = CustomGPModel(train_x, train_y, likelihood, num_categories=5)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    
    for i in range(num_iters):
        model.train()
        optimizer.zero_grad()
        loss = -model.likelihood(model(train_x)).log_prob(train_y).sum()
        loss.backward()
        optimizer.step()
        
        # UCB Acquisition
        model.eval()
        ucb = UpperConfidenceBound(model, beta=beta)
        new_x, _ = optimize_acqf(
            ucb,
            bounds=torch.tensor([[0., 0., 0., 0.], [1., 1., 1., 4.]]),
            q=1,
            num_restarts=5,
            raw_samples=20,
        )
        train_x = torch.cat([train_x, new_x])
        train_y = torch.cat([train_y, torch.tensor([0.0])])  # Placeholder y
        
    return train_x, train_y
